### Hoodnaming

This notebook converts the point names downloaded from WebCTRL into a list of pseudo point names for later use

In [27]:
import pandas as pd
import numpy as np
import re


In [61]:
pointnames = pd.read_csv("./building_csv/biotech_all.csv", skiprows=22)
# pointnames = pd.read_csv("./building_csv/baker_all.csv")

# pointnames = pd.read_csv("./building_csv/kimball_all.csv")
pointnames = pointnames.rename(columns={"Object ID": "ObjectId", "Device ID": "DeviceId", "Object Name": "ObjectName"})
pointnames.head()


,Location,Control Program,Name,Type,ObjectId,DeviceId,ObjectName,Path
0,/Cornell University/Biotech Server/Biotech/Bio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"Biotech ACS-1, 2, and 3 Performance",NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,AHU Exhaust Airflow,BAV,AV:16,DEV:1018006,ahu_exhaust_airflow_av_2,#biotech_acs-1_2_and_3_performance/ahu_exhaust...
3,NaN,NaN,AHU Outside Airflow,BAV,AV:17,DEV:1018006,ahu_outside_airflow_av_2,#biotech_acs-1_2_and_3_performance/ahu_outside...
4,NaN,NaN,AHU Supply Airflow,BAV,AV:18,DEV:1018006,ahu_supply_airflow_av_2,#biotech_acs-1_2_and_3_performance/ahu_supply_...


In [39]:
#convert rooms into an array: 
room_csv = pd.read_csv("./Rooms/biotech_room.csv")
# room_csv = pd.read_csv("./Rooms/kimball_room.csv")
roomArray = room_csv.iloc[:, 4].tolist()
print(roomArray)


['141', '143', '147B', '153', '153', '155', '160', '203', '203', '209', '209', '217', '221', '221', '227', '227', '233', '233', '236', '241', '241', '245', '245', '253', '253', '257', '257', '260', '260', '267', '267', '303', '303', '309', '309', '317', '317', '318', '321', '321', '327', '327', '333', '333', '336', '341', '341', '345', '353', '353', '357', '357', '360', '360', '367', '367', '403', '403', '409', '409', '417', '417', '418', '421', '421', '427', '427', '433', '436', '441', '441', '445', '445', '453', '453', '457', '457', '467', '467', 'G59', 'G54', '460B', '467', 'Room 3G49']


In [54]:
def find_occ_sens_fixed(df, room_list): 
    # Remove rows where any column contains the word "lighting" (case insensitive)
    lighting_mask = df.applymap(lambda x: isinstance(x, str) and "lighting" in x.lower()).any(axis=1)
    
    df_filtered = df[~lighting_mask].copy()  # Create a copy to avoid modifying the original DataFrame
    if "Type" in df_filtered.columns:
        df_filtered = df_filtered[df_filtered["Type"].str.fullmatch(r"(?i)bbi", na=False)]
    # Ensure the "Path" column is string format and not NaN
    df_filtered["Path"] = df_filtered["Path"].astype(str).fillna("")
    
    # # Filter rows where "Path" contains "occ" (case insensitive)
    occ_mask = df_filtered["Path"].str.contains(r"(?i)occ", na=False)
    df_filtered = df_filtered[occ_mask]

    # # Filter rows where "Path" contains "non fume hood lab spaces" (case insensitive)
    non_fume_mask = df_filtered["Path"].str.contains(r"(?i)non_fume_hood_lab_spaces", na=False)
    df_filtered = df_filtered[~non_fume_mask]

    # # Find exact matches for "Occ Sensor" in the "Name" column (case insensitive)
    exact_occ_sens_matches = df_filtered[df_filtered["Name"].str.fullmatch(r"(?i)(occ sensor|occupancy sensor)", na=False)].copy()
    # Save results to CSV
    exact_occ_sens_matches.to_csv("occ_sensor_matching_rooms.csv", index=False)

    # Print the filtered DataFrame
    print(exact_occ_sens_matches)

# Run the function using the room list
find_occ_sens_fixed(pointnames, roomArray)


                                                Location  \
231    /Cornell University/Biotech Server/Baker Lab/B...   
318    /Cornell University/Biotech Server/Baker Lab/B...   
437    /Cornell University/Biotech Server/Baker Lab/B...   
553    /Cornell University/Biotech Server/Baker Lab/B...   
669    /Cornell University/Biotech Server/Baker Lab/B...   
...                                                  ...   
23358  /Cornell University/Biotech Server/Baker Lab/T...   
23712  /Cornell University/Biotech Server/Baker Lab/T...   
24059  /Cornell University/Biotech Server/Baker Lab/T...   
24214  /Cornell University/Biotech Server/Baker Lab/T...   
25425  /Cornell University/Biotech Server/Baker Lab/F...   

            Control Program              Name Type ObjectId   DeviceId  \
231           B08 (VAV-BL1)  Occupancy Sensor  BBI     BI:1  DEV:30617   
318           B12 (PHX-BM1)  Occupancy Sensor  BBI     BI:2  DEV:30618   
437           B16 (VAV-BY1)  Occupancy Sensor  BBI     BI

/var/folders/dm/qcxk0by517nbn3hv449blw1c0000gn/T/ipykernel_3721/3455333809.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  lighting_mask = df.applymap(lambda x: isinstance(x, str) and "lighting" in x.lower()).any(axis=1)


In [64]:
def sash_pos_finder(df):
    # Filter for rows where "Name" is "Sash"
    df_filtered = df[df["Name"].str.contains(r"(?i)sash", na=False)].copy() 
    
    # filter for "BAI" in the type column 
    if "Type" in df_filtered.columns:
        df_filtered = df_filtered[df_filtered["Type"].str.fullmatch(r"(?i)(bai|asvi)", na=False)]

    # Save to CSV
    df_filtered.to_csv("sash.csv", index=False)
    
    # Print the filtered DataFrame
    print(df_filtered)

# Run the function using `pointnames`
sash_pos_finder(pointnames)


      Location Control Program                             Name Type ObjectId  \
2001       NaN             NaN    Fume Hood Sash Position Input  BAI     AI:3   
2192       NaN             NaN                    HOOD SASH POS  BAI     AI:4   
4792       NaN             NaN                    HOOD SASH POS  BAI     AI:4   
4839       NaN             NaN                    HOOD SASH POS  BAI     AI:4   
4914       NaN             NaN                    HOOD SASH POS  BAI     AI:4   
...        ...             ...                              ...  ...      ...   
22215      NaN             NaN                    HOOD SASH POS  BAI     AI:4   
22262      NaN             NaN                    HOOD SASH POS  BAI     AI:4   
22309      NaN             NaN                    HOOD SASH POS  BAI     AI:4   
22480      NaN             NaN  Fume Hood-1 Sash Position Input  BAI     AI:4   
22726      NaN             NaN  Fume Hood-1 Sash Position Input  BAI     AI:4   

          DeviceId       Ob

In [62]:
def find_zone_temp(df): 
    # Filter for rows where "Name" is "Sash"
    df_filtered = df[df["Name"].str.contains(r"(?i)zone temp", na=False)].copy() 
    
    # filter for "BAI" in the type column 
    if "Type" in df_filtered.columns:
        df_filtered = df_filtered[df_filtered["Type"].str.fullmatch(r"(?i)asvi", na=False)]

    # Save to CSV
    df_filtered.to_csv("zone_temp.csv", index=False)
    
    # Print the filtered DataFrame
    print(df_filtered)

# Run the function using `pointnames`
find_zone_temp(pointnames)

      Location Control Program       Name  Type ObjectId     DeviceId  \
1908       NaN             NaN  Zone Temp  ASVI     AI:1  DEV:1018418   
1954       NaN             NaN  Zone Temp  ASVI     AI:1  DEV:1018419   
2007       NaN             NaN  Zone Temp  ASVI     AI:5  DEV:1018362   
2123       NaN             NaN  Zone Temp  ASVI     AI:3  DEV:1018363   
2298       NaN             NaN  Zone Temp  ASVI     AI:3  DEV:1018364   
...        ...             ...        ...   ...      ...          ...   
26250      NaN             NaN  Zone Temp  ASVI     AI:3    DEV:49405   
26320      NaN             NaN  Zone Temp  ASVI     AI:3    DEV:49406   
26392      NaN             NaN  Zone Temp  ASVI     AI:3    DEV:49407   
26473      NaN             NaN  Zone Temp  ASVI     AI:3    DEV:49408   
27085      NaN             NaN  Zone Temp  ASVI     AI:1  DEV:1018507   

        ObjectName                                               Path  
1908   zone_temp_1                    #biotech_mr_g

In [47]:
#We want to collect the following:
#Air Flow Point, e.g: "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hoodvalve_flow/trend_log", HOOD FLOW

#Sash Opening Point, e.g: "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash", HOOD SASH POS

#Occupancy Point, e.g: "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend", OCC SENSOR

#Internal Temperature Point, e.g: "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log", ZONE TEMP

"""
The goal of this function is to output an organized collection of point names corresponding to specific fume hoods throughout the Biotechnology building. It is unclear the best path forward at the moment, however, we will try to start small and then expand upwards. There will probably be a lot of edge cases to handle, but we will see. Using regular expressions seems to have helped quite a bit.
"""




def csv_parser(df):
    pass
    #print(True and df["Point"].str.contains("_control"))
    # hood_flows = np.array(df[df["Pointname"]== "HOOD FLOW" ]["Point"])

    # zone_temps = df[df["Pointname"]== "ZONE TEMP"]["Point"]
    # zone_temps = [i for i in zone_temps if re.search("lab_[A-Za-z]*[0-9]+[A-Za-z]*_control/zonetemp",i)]
    ##biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log
    # zone_temps = [i.split("/")[:4] for i in zone_temps]
    # zone_temps = ['/'.join(i) for i in zone_temps]
    # zone_temps = [i + "/zone/zone_temp/trend_log" for i in zone_temps]

    # vals= {"Flow Sensor":hood_flows, "Sash":sash_pos}
    # odf = pd.DataFrame(data = vals)
    #display(ndf)
    # print(len(hood_flows))
    # print(len(sash_pos))
    # print(len(occ_sens))
    # print(len(zone_temps))

    # ovals= {"Occupancy":occ_sens, "Internal Temp":zone_temps}
    # ndf = pd.DataFrame(data = ovals)

    # return [ndf,odf]


      Location Control Program        Name Type ObjectId     DeviceId  \
438        NaN             NaN  Occ Sensor  BNI      NaN  DEV:1018003   
439        NaN             NaN  Occ Sensor  BNI      NaN  DEV:1018003   
440        NaN             NaN  Occ Sensor  BNI      NaN  DEV:1018003   
441        NaN             NaN  Occ Sensor  BNI      NaN  DEV:1018003   
442        NaN             NaN  Occ Sensor  BNI      NaN  DEV:1018003   
...        ...             ...         ...  ...      ...          ...   
23539      NaN             NaN  OCC SENSOR  BBI     BI:1    DEV:49326   
23590      NaN             NaN  OCC SENSOR  BBI     BI:1    DEV:49327   
23641      NaN             NaN  OCC SENSOR  BBI     BI:1    DEV:49444   
23692      NaN             NaN  OCC SENSOR  BBI     BI:1    DEV:49358   
23750      NaN             NaN  Occ Sensor  BBI     BI:1    DEV:49429   

         ObjectName                                               Path  
438             NaN  #biotech/biotech_building_ope

In [7]:
#Notice how the dataframes are not equal sizes; this is because certain labs are missing hood and sash points
#By looking at an excel file I found the following:
#Lab 147, Lab 217, Lab 218, Lab 427 are all missing hood and sash points
#We will remove these from the dataset

dfs = csv_parser(pointnames)
occ_temp = dfs[0]
hood_sash = dfs[1]
occ_temp = occ_temp.drop([3,9,11,58])
occ_temp = occ_temp.reset_index()
occ_temp = occ_temp.drop(["index"],axis=1)
occ_temp

                                          Flow Sensor  \
0   #biotech/biotech_ground_floor/ground_floor_fum...   
1   #biotech/biotech_1st_floor/first_floor_fume_ho...   
2   #biotech/biotech_1st_floor/first_floor_fume_ho...   
3   #biotech/biotech_1st_floor/first_floor_fume_ho...   
4   #biotech/biotech_1st_floor/first_floor_fume_ho...   
..                                                ...   
61  #biotech/biotech_4th_floor/fourth_floor_fume_h...   
62  #biotech/biotech_4th_floor/fourth_floor_fume_h...   
63  #biotech/biotech_4th_floor/fourth_floor_fume_h...   
64  #biotech/biotech_4th_floor/fourth_floor_fume_h...   
65  #biotech/biotech_4th_floor/fourth_floor_fume_h...   

                                                 Sash  
0   #biotech/biotech_ground_floor/ground_floor_fum...  
1   #biotech/biotech_1st_floor/first_floor_fume_ho...  
2   #biotech/biotech_1st_floor/first_floor_fume_ho...  
3   #biotech/biotech_1st_floor/first_floor_fume_ho...  
4   #biotech/biotech_1st_floor/firs

,Occupancy,Internal Temp
0,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...
1,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
2,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
3,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
4,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
...,...,...
61,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...
62,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...
63,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...
64,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...


In [15]:
#Now we can just combine the dataframes together and we can begin working on a hierarchical structure

combined = hood_sash.join(occ_temp)
display(combined)

,Flow Sensor,Sash,Occupancy,Internal Temp
0,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...
1,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
2,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
3,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
4,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
...,...,...,...,...
61,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...
62,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...
63,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...
64,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...


In [16]:
def get_floor(text):
    blocks = text.split("/")
    floor_block = blocks[1]
    floor = floor_block.split("_")[1]
    return floor

combined['Floor'] = [get_floor(i) for i in combined["Flow Sensor"]]
#combined

def get_lab(text):
    blocks = text.split("/")
    lab_block = blocks[3]
    lab = lab_block.split("_")[1]
    return lab

combined["Lab"] = [get_lab(i) for i in combined["Flow Sensor"]]
# replace the floor with one number/letter and for lab make it uniform
combined['Floor'] = combined['Floor'].str[:1].str.upper()
combined['Lab'] = combined['Lab'].str.upper()

# add building column 
def get_building(text):
    blocks = text.split("/")
    building_block = blocks[0]
    return building_block[1:]

combined['building'] = [get_building(i) for i in combined["Flow Sensor"]]

# add hood column (I left it as 1 for now put in manually)
combined['hood'] = "1"

# add name column UMM FIX THIS???
# def get_name(text):
#     return str(combined['building']) + "Floor_" + str(combined['Floor']) + "Lab_" + str(combined['Lab']) + ".Hood_" + "1"
# combined['name'] = [get_name(i) for i in combined]

combined

,Flow Sensor,Sash,Occupancy,Internal Temp,Floor,Lab,building,hood
0,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,G,G54,biotech,1
1,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,1,141,biotech,1
2,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,1,143,biotech,1
3,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,1,147B,biotech,1
4,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,1,153,biotech,1
...,...,...,...,...,...,...,...,...
61,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,4,445B,biotech,1
62,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,4,453,biotech,1
63,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,4,453B,biotech,1
64,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,4,457,biotech,1


In [17]:
#Rearranging the column orders :)


combined["External Temp"] = "#biotech/ground_flr_mech/building_hydronic_heating_syatems/reheat_heat_exchanger/oat"
combined["Server"] = "biotech_main"
cols = ["building","hood","Lab","Floor","Server", "Flow Sensor","Sash","Occupancy","Internal Temp","External Temp"]
combined = combined[cols]
#combined.to_csv('C:\\Users\\Rob\\Documents\\GitHub\\fume-hood-dashboard\\pointnames.csv')
combined

,building,hood,Lab,Floor,Server,Flow Sensor,Sash,Occupancy,Internal Temp,External Temp
0,biotech,1,G54,G,biotech_main,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/ground_flr_mech/building_hydronic_hea...
1,biotech,1,141,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...
2,biotech,1,143,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...
3,biotech,1,147B,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...
4,biotech,1,153,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...
...,...,...,...,...,...,...,...,...,...,...
61,biotech,1,445B,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...
62,biotech,1,453,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...
63,biotech,1,453B,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...
64,biotech,1,457,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...


In [18]:
df = combined.copy()
floor_map = {"ground":"Ground", "1st":"First","2nd":"Second", "3rd":"Third","4th":"Fourth"}
df["FloorNum"] = df["Floor"].map(floor_map)
display(df)

,building,hood,Lab,Floor,Server,Flow Sensor,Sash,Occupancy,Internal Temp,External Temp,FloorNum
0,biotech,1,G54,G,biotech_main,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
1,biotech,1,141,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
2,biotech,1,143,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
3,biotech,1,147B,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
4,biotech,1,153,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
61,biotech,1,445B,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
62,biotech,1,453,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
63,biotech,1,453B,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
64,biotech,1,457,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN


In [19]:
#Biotech.floor_num.lab_num.hood_num.{pointType}
#4 Point Types:[energy.occ,energy.unocc,sashOpenTime.occ, sashOpenTime.unocc]

def point_maker(lab,floor,hood):
    nstr = "Biotech." + str(floor) + "Floor" + ".Lab_" +  str(lab)   + ".Hood_" + str(hood) + "."
    nlist = [nstr+"energy.occ",nstr+"energy.unocc",nstr+"sashOpenTime.occ",nstr+"sashOpenTime.unocc"]
    return nlist



#display(df)

# test_case = point_maker(141,1,1)
# test_case

def make_all_points(df):
    energy_occ = []
    energy_unocc = []
    sashOpenTime_occ = []
    sashOpenTime_unocc = []
    for i in range(len(df["FloorNum"])):
        response = point_maker(df["Lab"][i],df["FloorNum"][i],1)
        energy_occ.append(response[0])
        energy_unocc.append(response[1])
        sashOpenTime_occ.append(response[2])
        sashOpenTime_unocc.append(response[3])
    df["Energy Occupied"] = energy_occ
    df["Energy Unoccupied"] = energy_unocc
    df["Sash Open Time Occupied"] = sashOpenTime_occ
    df["Sash Open Time Unoccupied"] = sashOpenTime_unocc
    return df

ndf = make_all_points(df)
display(ndf)







,building,hood,Lab,Floor,Server,Flow Sensor,Sash,Occupancy,Internal Temp,External Temp,FloorNum,Energy Occupied,Energy Unoccupied,Sash Open Time Occupied,Sash Open Time Unoccupied
0,biotech,1,G54,G,biotech_main,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_G54.Hood_1.energy.occ,Biotech.nanFloor.Lab_G54.Hood_1.energy.unocc,Biotech.nanFloor.Lab_G54.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_G54.Hood_1.sashOpenTime.u...
1,biotech,1,141,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_141.Hood_1.energy.occ,Biotech.nanFloor.Lab_141.Hood_1.energy.unocc,Biotech.nanFloor.Lab_141.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_141.Hood_1.sashOpenTime.u...
2,biotech,1,143,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_143.Hood_1.energy.occ,Biotech.nanFloor.Lab_143.Hood_1.energy.unocc,Biotech.nanFloor.Lab_143.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_143.Hood_1.sashOpenTime.u...
3,biotech,1,147B,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_147B.Hood_1.energy.occ,Biotech.nanFloor.Lab_147B.Hood_1.energy.unocc,Biotech.nanFloor.Lab_147B.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_147B.Hood_1.sashOpenTime....
4,biotech,1,153,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_153.Hood_1.energy.occ,Biotech.nanFloor.Lab_153.Hood_1.energy.unocc,Biotech.nanFloor.Lab_153.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_153.Hood_1.sashOpenTime.u...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,biotech,1,445B,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_445B.Hood_1.energy.occ,Biotech.nanFloor.Lab_445B.Hood_1.energy.unocc,Biotech.nanFloor.Lab_445B.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_445B.Hood_1.sashOpenTime....
62,biotech,1,453,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_453.Hood_1.energy.occ,Biotech.nanFloor.Lab_453.Hood_1.energy.unocc,Biotech.nanFloor.Lab_453.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_453.Hood_1.sashOpenTime.u...
63,biotech,1,453B,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_453B.Hood_1.energy.occ,Biotech.nanFloor.Lab_453B.Hood_1.energy.unocc,Biotech.nanFloor.Lab_453B.Hood_1.sashOpenTime.occ,Biotech.nanFloo

In [20]:
ndf.to_csv("biotech_point_names.csv")